# commander examples


In [1]:
from typing import List
from math import pi
import rospy
from copy import deepcopy

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,

    SetEe,
)
from commander.utils import poses_from_yaml, load_scene
from commander.transform_utils import orient_poses, create_rotation_matrix, apply_transformation

plan_goal_srv = rospy.ServiceProxy('commander/plan_goal', PlanGoal)
plan_sequence_srv = rospy.ServiceProxy('commander/plan_sequence', PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy('commander/execute_trajectory', ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy('commander/get_tcp_pose', GetTcpPose)
set_ee_srv = rospy.ServiceProxy('commander/set_ee', SetEe)
pick_place_srv = rospy.ServiceProxy('commander/pick_place', PickPlace)


def display_poses(poses: List[Pose], frame_id: str = 'base_link') -> None:
    rospy.wait_for_service('/visualize_poses', timeout=10)
    visualize_poses = rospy.ServiceProxy('/visualize_poses', VisualizePoses)
    visualize_poses(frame_id, poses)


rospy.init_node('robot_program')
scene = PlanningSceneInterface()
load_scene()

In [2]:

cam_home = [0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0]

# the following args for planner are available:
# - 'ompl' will use the ompl planning pipeline with RTTConnect planner
# - 'ptp' will use the pilz planning pipeline with the PTP planner
# - 'lin' will use the pilz planning pipeline with the LIN planner

plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))
success = execute_trajectory_srv()

success = set_ee_srv('rgb_camera_tcp')

target0 = Pose(
    position=Point(0.5, -0.4, 0.4),
    orientation=Quaternion(0.7071067811865476, -0.7071067811865475, 0.0, 0.0),
)
target1 = Pose(
    position=Point(0.5, 0.4, 0.4),
    orientation=Quaternion(0.7071067811865476, -0.7071067811865475, 0.0, 0.0),
)
display_poses([target0, target1])
success = plan_goal_srv(
    Goal(
        pose=target0,
        vel_scale=0.2,
        acc_scale=0.2,
        planner='ptp',
    )
).success
success = execute_trajectory_srv()
success = plan_goal_srv(
    Goal(
        pose=target1,
        vel_scale=0.2,
        acc_scale=0.2,
        planner='lin',
    )
).success

success = execute_trajectory_srv()

plan_sequence_srv(
    goals=[
        Goal(
            joint_values=[0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0],
            vel_scale=0.2,
            acc_scale=0.2,
            planner='ptp',
        ),
        Goal(pose=target0, vel_scale=0.2, acc_scale=0.2, planner='ptp'),
        Goal(pose=target1, vel_scale=0.2, acc_scale=0.2, planner='lin'),
    ],
    blends=[0.0, 0.2, 0.0],
)
success = execute_trajectory_srv()

sequence = poses_from_yaml('/dev_ws/src/software_II_project/commander/data/sequence_S.yaml')
origin = Pose(position=Point(0.75, 0.0, -0.6), orientation=Quaternion(0.0, 0.0, 0.0, 1.0))
sequence = orient_poses(origin, sequence)
tf = create_rotation_matrix([0, 0, pi])
sequence = [apply_transformation(pose, tf) for pose in sequence]
display_poses(sequence)
success = set_ee_srv('camera_tcp')
success = plan_goal_srv(
    Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp')
).success

success = execute_trajectory_srv()
for pose in sequence:
    resp = plan_goal_srv(Goal(pose=pose, vel_scale=0.2, acc_scale=0.2, planner='ptp'))
    if resp.success:
        success = execute_trajectory_srv()
    else:
        break



In [3]:
cam_home = [0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0]
plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))
success = execute_trajectory_srv()

In [4]:

success = set_ee_srv('rgb_camera_tcp')

pick_pose = Pose(
    position=Point(-0.8, 0.0, -0.5),
    orientation=Quaternion(-0.7071067811865476, -0.7071067811865475, 0, 0),
)

place_pose = Pose(
    position=Point(0.0, -0.8, -0.5),
    orientation=Quaternion(-0.7071067811865476, 0, -0.7071067811865475, 0),
)

pick_approach = deepcopy(pick_pose)
pick_approach.position.z += 0.1
obj_pose = PoseStamped()
thickness = 0.1
obj_pose.header.frame_id = 'base_link'
obj_pose.pose = deepcopy(pick_pose)
obj_pose.pose.position.z -= (thickness / 2.0) + 0.05
display_poses([pick_pose])
scene.add_box('plank', obj_pose, (0.5, 0.1, thickness))

success = plan_goal_srv(
    Goal(
        pose=pick_pose,
        vel_scale=0.2,
        acc_scale=0.2,
        planner='ptp',
    )
).success
success = execute_trajectory_srv()

pick_place_srv(True, 'plank')

cam_home = [0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0]
plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))
success = execute_trajectory_srv()

display_poses([place_pose])

success = plan_goal_srv(
    Goal(
        pose=place_pose,
        vel_scale=0.2,
        acc_scale=0.2,
        planner='ptp',
    )
).success
success = execute_trajectory_srv()

pick_place_srv(False, 'plank')

success: True